### Origional KSA algorithm 
```
sort array S based on a secret key

for i from 0 to 255
    S[i] := i
endfor
j := 0
for i from 0 to 255
    j := (j + S[i] + key[i mod keylength]) mod 256
    swap values of S[i] and S[j]
endfor
``` 

### two stage KSA Algorithm: 

```





In [5]:
import math

def KSA(key,s):
    j = 0
    length = len(s)
    for i in range(length):
        j = (j+s[i]+key[i%len(key)])% length
        s[i],s[j] = s[j],s[i] #list swap   
    return s

#mix two arrays of size N to create a child array of size N/2 with no correlation to its parents
#arguments: key: list<int>, length: int (internal state array size)
def mix(key1,key2,length):
    s1 = KSA(key1,list(range(length)))
    s2 = KSA(key2,list(range(length)))
    s3 = list(range(length))
    
    #pick only even numbers to half the size of S
    for i in range(length): 
        if(i % 2 == 0):
            '''mutating the numbers to get a semi random set of numbers instead of the predictable 0-255'''
            s3[i] = (s2[i] + s3[i] ^ i) % length 
    return s3

In [6]:
def PRGA(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % len(S)
        j = (j + S[i]) % len(S)
        S[i], S[j] = S[j], S[i]  # swap

        K = S[(S[i] + S[j]) % len(S)]
        yield K


In [7]:
def RC4_2S(k1,k2):    
    #create internal secret state S, that is composed of 2 seperate states s1 and s2. 
    S = mix(k1,k2,512)
    keystream = PRGA(S) #return generator that constantly updates the secret internal state S
    #discard step: The first 1536 bytes of keystream generated by the cipher MUST be discarded to meet RCFOUR256 spec
    for i in range(1536): 
        _discardedbyte = next(keystream)
    return keystream

In [8]:
#bitwise xor in python is performed on normal integers. 
def xor(text,keystream): 
    result = []
    for c in text: 
        try: 
            c = ord(c) #convert chr to dec
        except: 
            pass         
        result.append( c ^ next(keystream) ) 
    return result  

In [9]:
k1 = list(b"of the baskervilles")
k2 = list(b"Hound") 
keystream = RC4_2S(k1,k2)
#encrypt:
ciphertext = xor('Sherlock Holmes',keystream)
 
print(ciphertext)
#format integers to hex for comparison.
for c in ciphertext:
    print("%02X" % (c),end='')
    #CC86A411790112174E2193514361D21A678

[204, 134, 164, 279, 144, 274, 372, 226, 403, 81, 67, 97, 210, 422, 120]
CC86A411790112174E2193514361D21A678

In [10]:
#decrypt:
keystream = RC4_2S(k1,k2)
plaintext = xor(ciphertext,keystream)
for c in plaintext: 
    print(chr(c),end='')

Sherlock Holmes